# Car Price Prediction - Web Scraping Part

In [5]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from bs4 import BeautifulSoup as bs
import requests
import re
import time
import numpy as np

In [6]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [19]:
url1 = 'https://www.cars.com/shopping/results/\
?page='
url2 = '&page_size=20&list_price_max=&makes[]=\
&maximum_distance=all&models[]=&stock_type=used&zip='

url3 = 'https://www.cars.com/shopping/results/\
?page='
url4 = '&page_size=20&list_price_max=&makes[]=&maximum_distance=all&models[]=&stock_type=new&zip='

url_list = []

for i in range(60):
    url_list.append(url1 + str(i+1) + url2)

column_names = ['name', 'price', 'mileage', 'ex_color', 'in_color', 'drivetrain',\
                'mpg', 'fuel_type', 'transmission', 'engine']    

df = pd.DataFrame(columns = column_names)

response_list = []

tmp_df = pd.DataFrame()

for url in url_list:
    ex_colors = []
    in_colors = []
    drivetrains = []
    mpgs = []
    fuel_types = []
    transmissions = []
    engines = []
    res = requests.get(url).text
    soup = bs(res)
    
    # getting names
    names = soup.find_all(name='h2', attrs={'class': 'title'})
    names = pd.DataFrame(names)
    tmp_df['name'] = names

    # getting prices
    prices = soup.find_all(name='span', attrs={'class': 'primary-price'})
    prices = pd.DataFrame(prices)
    prices = prices.replace('\D', '', regex=True)
    tmp_df['price'] = prices

    # getting mileages
    mileages = soup.find_all(name='div', attrs={'class': 'mileage'})
    mileages = pd.DataFrame(mileages)
    mileages = prices.replace('\D', '', regex=True)
    tmp_df['mileage'] = mileages

    # getting links
    link_list = soup.find_all(name='a', attrs={'class': 'vehicle-card-link js-gallery-click-link'})
    links = []
    for link in link_list:
        links.append('https://www.cars.com' + link.get('href'))
        

    for link in links:
        res = requests.get(link).text
        time.sleep(2)
        sp = bs(res)
        # adding exterior color
        ex_color = sp.find(name='dt', text='Exterior color')\
            .next.next.next.text
        ex_colors.append(ex_color)

        # adding interior color
        in_color = sp.find(name='dt', text='Interior color')\
            .next.next.next.text
        in_colors.append(in_color)

        # adding drivetrain
        dt = sp.find(name='dt', text='Drivetrain').next.next.next.text
        drivetrains.append(dt)

        # adding mpg
        spf = sp.find(name='a', attrs={'class': 'sds-tooltip__trigger'})
        if spf != None:
            mpg = spf.previous_sibling.previous_sibling.text
            mpgs.append(mpg)
        else:
            mpgs.append('None')

        # adding fuel type
        fuel_type = sp.find(name='dt', text='Fuel type').next.next.next.text
        fuel_types.append(fuel_type)

        # adding transmission
        transmission = sp.find(name='dt', text='Transmission')\
            .next.next.next.text
        transmissions.append(transmission)

        # adding engine
        engine = sp.find(name='dt', text='Engine').next.next.next.text
        engines.append(engine)
        
    tmp_df['ex_color'] = ex_colors
    tmp_df['in_color'] = in_colors
    tmp_df['drivetrain'] = drivetrains
    tmp_df['mpg'] = mpgs
    tmp_df['fuel_type'] = fuel_types
    tmp_df['transmission'] = transmissions
    tmp_df['engine'] = engines
    
    # df.append(tmp_df)
    
    df = pd.concat([df, tmp_df], axis=0)
    
    tmp_df.drop(tmp_df.index,inplace=True)

df = df.reset_index()

# response1 = requests.get(url1 + '1' + url2)

# soup = bs(response_list[0])

In [21]:
df.to_csv('car_info.csv', encoding='utf-8')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1193 entries, 0 to 1192
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         1193 non-null   int64 
 1   name          1193 non-null   object
 2   price         1193 non-null   object
 3   mileage       1193 non-null   object
 4   ex_color      1193 non-null   object
 5   in_color      1193 non-null   object
 6   drivetrain    1193 non-null   object
 7   mpg           1193 non-null   object
 8   fuel_type     1193 non-null   object
 9   transmission  1193 non-null   object
 10  engine        1193 non-null   object
dtypes: int64(1), object(10)
memory usage: 102.6+ KB
